# Chapter 11
## Section: Practicing with alibi_detect for drift detection

In [ ]:
!pip install alibi_detect
!pip install lightgbm

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from alibi_detect.cd import KSDrift
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score as bacc

# Generate synthetic data
X, y = make_classification(n_samples=10000, n_features=10, n_classes=2, random_state=42)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a LightGBM classifier on the training data
train_data = lgb.Dataset(X_train, label=y_train)
params = {
    "objective": "binary",
    "metric": "binary_logloss",
    "boosting_type": "gbdt"
}
clf = lgb.train(train_set = train_data, params = params, num_boost_round=100)

# Predict on the test set
y_pred = clf.predict(X_test)
y_pred = [1 if iter > 0.5 else 0 for iter in y_pred]

# Calculate the balanced accuracy of the predictions
balanced_accuracy = bacc(y_test, y_pred)
print('Balanced accuracy on the synthetic test set: {}'.format(balanced_accuracy))

# Create a DataFrame from the test data and predictions
df = pd.DataFrame(X_test, columns=[f"feature_{i}" for i in range(10)])
df["actual"] = y_test
df["predicted"] = y_pred

# Initialize the KSDrift detector
drift_detector = KSDrift(X_train)

# Calculate the drift scores and p-values
drift_scores = drift_detector.predict(X_test)
p_values = drift_detector.predict(X_test, return_p_val=True)

# Print the drift scores and p-values
print("Drift scores:")
print(drift_scores)
print("P-values:")
print(p_values)

[LightGBM] [Info] Number of positive: 4009, number of negative: 3991
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001019 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 8000, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.501125 -> initscore=0.004500
[LightGBM] [Info] Start training from score 0.004500
Balanced accuracy on the synthetic test set: 0.9479665316987853
Drift scores:
{'data': {'is_drift': 0, 'distance': array([0.02825 , 0.024625, 0.0225  , 0.01275 , 0.014   , 0.017125,
       0.01775 , 0.015125, 0.021375, 0.014625], dtype=float32), 'p_val': array([0.15258548, 0.28180763, 0.38703775, 0.95421314, 0.907967  ,
       0.72927415, 0.68762517, 0.8520056 , 0.45154762, 0.87837887],
      dtype=float32), 'threshold': 0.005}, 'meta': {'name': 'KSDrift', 'online': False, 'data_type': None, 'version': 